In [0]:
# Choose a unique sandbox (change if you re-run)
catalog   = "main" # or your UC catalog
schema    = "demo_schema_migration"
table     = "bronze_customers_demo"
base_path = f"/tmp/{table}"

raw_path        = f"{base_path}/raw"
schema_loc_path = f"{base_path}/_schemas"
delta_path      = f"{base_path}/delta"

spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")
spark.sql(f"CREATE SCHEMA  IF NOT EXISTS {catalog}.{schema}")
spark.sql(f"USE {catalog}.{schema}")

# Clean up previous runs
for p in [raw_path, schema_loc_path, delta_path]:
    if any([f.path == p for f in dbutils.fs.ls("/")]):
        dbutils.fs.rm(p, True)
